# kNN Ingredient Replacement
Replace an ingredient with its k nearest neighbors in the USDA ingreident nutrition adjacency matrix. In theory, ingredients should be replaced by ones that have similar nutritional values.

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [17]:
nutrition_adj_mat = np.load('../data/Adjacency_Matrix_5.npy')
combined_adj_mat = np.load('../data/Adjacency_Matrix_9.npy')
ingredient_list = np.load('../data/Ingredient_List_USDA_Nutrition_Info.npy')

In [5]:
def suggestKBestReplacements(nutrition_adj_mat,ingr_number,k):
    adj_values = nutrition_adj_mat[ingr_number,:]
    maxvals = idx = (-adj_values).argsort()[:k]
    if (np.all(adj_values == 0)):
        maxvals = []
    return maxvals

In [6]:
ingr_number = 25 #Change this number to try different ingredients! 
k = 5
maxvals = suggestKBestReplacements(nutrition_adj_mat,ingr_number,k)
print("Original Ingredient: {0}".format(ingredient_list[ingr_number]))
if (len(maxvals)==0):
    print("No replacements found!")
else: 
    print("{0} best replacements: {1}".format(k,ingredient_list[maxvals]))

Original Ingredient: beef, rib eye steak, boneless, lip off, separable lean and fat, trimmed to 0" fat, all grades, cooked, grilled
5 best replacements: ['fish, catfish, channel, wild, raw' 'fish, tilapia, raw'
 'cheese, mozzarella, low sodium' 'meatballs, meatless' 'cheese, brie']


Try with the combined recipe/nutrition matrix

In [20]:
ingr_number = 27 #Change this number to try different ingredients! 
k = 5
maxvals = suggestKBestReplacements(combined_adj_mat,ingr_number,k)
print("Original Ingredient: {0}".format(ingredient_list[ingr_number]))
if (len(maxvals)==0):
    print("No replacements found!")
else: 
    print("{0} best replacements: {1}".format(k,ingredient_list[maxvals]))

Original Ingredient: beverages, almond milk, unsweetened, shelf stable
5 best replacements: ['beverages, ovaltine, classic malt powder'
 'beverages, gerolsteiner brunnen gmbh & co. kg,gerolsteiner naturally sparkling mineral water,'
 'soymilk, original and vanilla, unfortified'
 'milk, fluid, 1% fat, without added vitamin a and vitamin d'
 'yogurt, greek, plain, nonfat']
